In [1]:
import getopt
import pandas as pd
import numpy as np
import json
import sklearn
import sklearn.ensemble
import sklearn.neural_network

import sklearn.neighbors
import pickle
import script as sc

In [2]:
import os,sys,inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,currentdir)

In [3]:
def get_lags(data, monitor, lags):
    if monitor == "actigraph_wrist":
        prev_data = data[['mean.vm', 'sd.vm', 'mean.ang', 'sd.ang', 'p625', 'dfreq', 'ratio.df']]
    elif monitor == "actigraph_wrist23":
        prev_data = data[['mean.vm', 'sd.vm', 'mean.ang', 'sd.ang', 'p625', 'dfreq',
                          'ratio.df', 'min.x', 'min.y', 'min.z', 'max.x', 'max.y',
                          'max.z', 'mean.x', 'mean.y', 'mean.z', 'sd.x', 'sd.y',
                          'sd.z', 'mean.xy', 'mean.yz', 'mean.xz', 'mean.xyz']]
    elif monitor == "biostamp_thigh":
        prev_data = data[['thigh.mean.vm', 'thigh.sd.vm', 'thigh.mean.ang', 'thigh.sd.ang', 'thigh.p625', 'thigh.dfreq',
                          'thigh.ratio.df', 'thigh.min.x', 'thigh.min.y', 'thigh.min.z', 'thigh.max.x', 'thigh.max.y',
                          'thigh.max.z', 'thigh.mean.x', 'thigh.mean.y', 'thigh.mean.z', 'thigh.sd.x', 'thigh.sd.y',
                          'thigh.sd.z', 'thigh.mean.xy', 'thigh.mean.yz', 'thigh.mean.xz', 'thigh.mean.xyz']]
    elif monitor == "biostamp_chest":
        prev_data = data[['chest.mean.vm', 'chest.sd.vm', 'chest.mean.ang', 'chest.sd.ang', 'chest.p625', 'chest.dfreq',
                          'chest.ratio.df', 'chest.min.x', 'chest.min.y', 'chest.min.z', 'chest.max.x', 'chest.max.y',
                          'chest.max.z', 'chest.mean.x', 'chest.mean.y', 'chest.mean.z', 'chest.sd.x', 'chest.sd.y',
                          'chest.sd.z', 'chest.mean.xy', 'chest.mean.yz', 'chest.mean.xz', 'chest.mean.xyz']]
    elif monitor == "biostamp_thigh_chest":
        prev_data = data[['thigh.mean.vm', 'thigh.sd.vm', 'thigh.mean.ang', 'thigh.sd.ang', 'thigh.p625','thigh.dfreq',
                      'thigh.ratio.df', 'thigh.min.x', 'thigh.min.y', 'thigh.min.z', 'thigh.max.x', 'thigh.max.y',
                      'thigh.max.z', 'thigh.mean.x', 'thigh.mean.y', 'thigh.mean.z', 'thigh.sd.x', 'thigh.sd.y',
                      'thigh.sd.z', 'thigh.mean.xy', 'thigh.mean.yz', 'thigh.mean.xz', 'thigh.mean.xyz',
                      'chest.mean.vm', 'chest.sd.vm', 'chest.mean.ang', 'chest.sd.ang', 'chest.p625', 'chest.dfreq',
                      'chest.ratio.df', 'chest.min.x', 'chest.min.y', 'chest.min.z', 'chest.max.x', 'chest.max.y',
                      'chest.max.z', 'chest.mean.x', 'chest.mean.y', 'chest.mean.z', 'chest.sd.x',  'chest.sd.y',
                      'chest.sd.z', 'chest.mean.xy', 'chest.mean.yz', 'chest.mean.xz', 'chest.mean.xyz']]
    elif monitor == "biostamp_thigh_chest7":
        prev_data = data[['thigh.mean.vm', 'thigh.sd.vm', 'thigh.mean.ang', 'thigh.sd.ang', 'thigh.p625', 'thigh.dfreq',
                          'thigh.ratio.df', 'chest.mean.vm', 'chest.sd.vm', 'chest.mean.ang', 'chest.sd.ang', 'chest.p625',
                          'chest.dfreq', 'chest.ratio.df']]
    elif monitor == "biostamp_thigh7":
        prev_data = data[['thigh.mean.vm', 'thigh.sd.vm', 'thigh.mean.ang', 'thigh.sd.ang', 'thigh.p625', 'thigh.dfreq',
                          'thigh.ratio.df']]
    elif monitor == "biostamp_chest7":
        prev_data = data[['chest.mean.vm', 'chest.sd.vm', 'chest.mean.ang', 'chest.sd.ang',
                          'chest.p625', 'chest.dfreq', 'chest.ratio.df']]

    for i in range(lags):
        copy = prev_data.copy(deep=True)
        copy.loc[-1] = copy.loc[0]
        copy.index = copy.index + 1  # shifting index
        copy.sort_index(inplace=True)
        copy.columns = 'last.' + str(i+1) + "." + copy.columns
        prev_data = copy
        data = pd.concat([data, copy], axis = 1)
        data = data.drop(data.index[len(data)-1])
    return data

In [4]:
# helper_functions
def save_pkl(classifier, filename):
    with open(filename, 'wb') as f:
        pickle.dump(classifier, f)
        print("Classifier saved as " + filename)

## functions for the various models -- these take in the training set
## and return the fitted model

def random_forest(trainX, trainY):
    rf = sklearn.ensemble.RandomForestClassifier(max_depth=15, random_state=0)
    rf.fit(trainX, trainY)
    return rf

In [49]:
model = "rf"
user = "Sam"
input_file = "tier_wrist_complete23.csv"
monitor = "actigraph_wrist23"

In [50]:
data = pd.DataFrame(sc.get_complete(user,input_file))
data = get_lags(data, monitor, 1)
data = data[data['transition'] != 1]
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'start.time', 'primary_behavior', 'primary_posture',
                 'primary_upperbody', 'primary_intensity', 'secondary_behavior', 'secondary_posture',
                 'secondary_upperbody', 'secondary_intensity', 'num_postures', 'transition', 'actual_time',
                 'coding'], axis=1)

# data has type, posture_coding and posture1, posture2

In [51]:
data[data['posture_coding'] == 'private/not coded']

,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,min.x,min.y,min.z,...,last.1.mean.x,last.1.mean.y,last.1.mean.z,last.1.sd.x,last.1.sd.y,last.1.sd.z,last.1.mean.xy,last.1.mean.yz,last.1.mean.xz,last.1.mean.xyz


In [52]:
# core data has type, core_posture, and posture_coding
core_data = data
core_y_var = 'posture1' # trying to predict core posture

# for classifying stand vs stand + move
stand_data = data[(data['posture1'] == 'LA- stand')]
stand_data = stand_data.drop(['posture1'], axis =1)
stand_y_var = 'posture2'

# for classifying walking vs stairs vs running
walk_data = data[(data['posture1'] == 'WA- walk')]
walk_data = walk_data.drop(['posture1'], axis=1)
walk_y_var = 'posture2'

# stand and move intensities
sm_data = stand_data[stand_data['posture2'] == 'LA- stand and move']
sm_data = sm_data.drop(['posture2'], axis=1) # don't need core posture
sm_y_var = 'posture_coding' # trying to predict posture_coding

# walking intensities
walk_intensity_data = walk_data[walk_data['posture2'] == 'WA- walk']
walk_intensity_data = walk_intensity_data.drop(['posture2'], axis=1)
walk_intensity_y_var = 'posture_coding'

data_with_predictions = pd.DataFrame()
coredata_with_predictions = pd.DataFrame()

In [53]:
for person in np.unique(data['type']):
    print(person)

    ## split training and testing by one person's observation
    core_train = core_data[core_data['type'] != person].copy(deep=True) # training set is everyone BUT this person
    core_test = core_data[core_data['type'] == person].copy(deep=True) # test on this person

    core_train_x = core_train.drop(['type', 'posture1', 'posture2', 'posture_coding'], axis=1) # drop extra columns
    core_train_y = core_train[core_y_var] # posture1's for training set

    stand_train = stand_data[stand_data['type'] != person].copy(deep=True)
    stand_train_x = stand_train.drop(['type', 'posture2', 'posture_coding'], axis=1)
    stand_train_y = stand_train[stand_y_var] # posture2's for stand training set
    
    walk_train = walk_data[walk_data['type'] != person].copy(deep=True)
    walk_train_x = walk_train.drop(['type', 'posture2', 'posture_coding'], axis=1)
    walk_train_y = walk_train[walk_y_var] # core_postures for walk training set
    
    sm_train = sm_data[sm_data['type'] != person].copy(deep=True) # training set is everyone BUT this person
    sm_train_x = sm_train.drop(['type', 'posture_coding'], axis=1) # drop extra columns
    sm_train_y = sm_train[sm_y_var]  # posture_codings for s+m training set

    walk_intensity_train = walk_intensity_data[walk_intensity_data['type'] != person].copy(deep=True)
    walk_intensity_train_x = walk_intensity_train.drop(['type', 'posture_coding'], axis=1)
    walk_intensity_train_y = walk_intensity_train[walk_intensity_y_var]  # posture_codings for walking training set

    core_classifier = None
    stand_classifier = None
    walk_classifier = None
    sm_classifier = None
    walk_intensity_classifier = None

    if model == "gradient_boosting":
        core_classifier = gradient_boost(core_train_x, core_train_y) # give classifier everything BUT this person
    elif model == "svm":
        core_classifier = svm(core_train_x, core_train_y)
    elif model == "knn":
        core_classifier = knn(core_train_x, core_train_y)
    elif model == "rf":
        core_classifier = random_forest(core_train_x, core_train_y)
        stand_classifier = random_forest(stand_train_x, stand_train_y)
        walk_classifier = random_forest(walk_train_x, walk_train_y)
        sm_classifier = random_forest(sm_train_x, sm_train_y)
        walk_intensity_classifier = random_forest(walk_intensity_train_x, walk_intensity_train_y)
    elif model == "neural_network":
        core_classifier = neural_net(core_train_x, core_train_y)
    else:
        print("no model chosen")
        exit(1)
        
    
    core_test_x = core_test.drop(['type', 'posture1', 'posture2', 'posture_coding'], axis=1) # drop extra columns
    core_test_y = core_test[core_y_var]

    core_pred = core_classifier.predict(core_test_x) # now try to predict this person
    core_test['predicted'] = core_pred
    corr = core_test['predicted'] == core_test['posture1']

    print(str(person + " tier 1 accuracy = "), sum(corr) / len(corr))
    
    coredata_with_predictions = coredata_with_predictions.append(core_test)
    
    # 2nd tier
    
    # predict stand vs stand and move
    stand_test = core_test[core_test['predicted'] == 'LA- stand']
    if (stand_test.shape[0] > 0):
        stand_test_x = stand_test.drop(['type', 'posture1', 'posture2', 'predicted', 'posture_coding'], axis=1)
        stand_pred = stand_classifier.predict(stand_test_x)
        stand_test['predicted'] = stand_pred
        
    # predict walk vs stairs vs run
    walk_test = core_test[core_test['predicted'] == 'WA- walk']
    if (walk_test.shape[0] > 0):
        walk_test_x = walk_test.drop(['type', 'posture1', 'posture2', 'predicted', 'posture_coding'], axis=1) # drop extra columns
        walk_pred = walk_classifier.predict(walk_test_x)
        walk_test['predicted'] = walk_pred
    
    
    # last tier
    
    # predict stand and move intensity
    sm_test = stand_test[stand_test['predicted'] == 'LA- stand and move'] # test on this person's stand and move
    if (sm_test.shape[0] > 0):
        sm_test_x = sm_test.drop(['type', 'posture1', 'posture2', 'predicted', 'posture_coding'], axis=1) # drop extra columns
        sm_pred = sm_classifier.predict(sm_test_x)
        sm_test['predicted'] = sm_pred
    
    # predict walking intensity
    walk_intensity_test = walk_test[walk_test['predicted'] == 'WA- walk'] # test on this person's stand and move
    if (walk_intensity_test.shape[0] > 0):
        walk_intensity_test_x = walk_intensity_test.drop(['type', 'posture1', 'posture2', 'predicted', 'posture_coding'], axis=1) # drop extra columns
        walk_intensity_pred = walk_intensity_classifier.predict(walk_intensity_test_x)
        walk_intensity_test['predicted'] = walk_intensity_pred
    
    
    pure_core_test = core_test[(core_test['predicted'] != 'LA- stand') & (core_test['predicted'] != 'WA- walk')]
    not_walk_test = walk_test[(walk_test['predicted'] != 'WA- walk')]
    pure_stand_test = stand_test[(stand_test['predicted'] != 'LA- stand and move')]
    
    person_test = pd.DataFrame()
    person_test = pure_core_test.append(not_walk_test)
    person_test = person_test.append(pure_stand_test)
    person_test = person_test.append(sm_test)
    person_test = person_test.append(walk_intensity_test)
    
    corr2 = person_test['predicted'] == person_test['posture_coding']
    
    print(str(person + " accuracy = "), sum(corr2) / len(corr2))
    
    data_with_predictions = data_with_predictions.append(person_test)
    
correct = data_with_predictions['posture_coding'] == data_with_predictions['predicted']

print ("Overall Model Accuracy = ",sum(correct) / len(correct))

A-11


/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/si

A-11 tier 1 accuracy =  0.6012731481481481


/Users/samhsu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/samhsu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/samhsu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

A-11 accuracy =  0.4262152777777778
A-12


...

W-4 accuracy =  0.269532384548359
W-9


/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/si

W-9 tier 1 accuracy =  0.7266187050359713


/Users/samhsu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/samhsu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/samhsu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

W-9 accuracy =  0.7266187050359713
Overall Model Accuracy =  0.6121096284346561


In [54]:
core_correct = coredata_with_predictions['posture1'] == coredata_with_predictions['predicted']

In [55]:
print("Core Model Accuracy = ", sum(core_correct) / len(core_correct))

Core Model Accuracy =  0.6924021112065791


In [56]:
coredata_with_predictions.to_csv("core_"+monitor+"_"+str(model)+"_"+input_file,index = False)

In [57]:
core_classifier = random_forest(data.drop(['type', 'posture_coding', 'posture1', 'posture2'], axis=1), data[core_y_var])

/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [58]:
save_pkl(core_classifier, "tier_wrist_core.sav")

data_with_predictions.to_csv(monitor+"_"+str(model)+"_"+input_file,index = False)

Classifier saved as tier_wrist_core.sav


In [59]:
stand_classifier = random_forest(stand_data.drop(['type', 'posture_coding', 'posture2'], axis=1), stand_data[stand_y_var])
walk_classifier = random_forest(walk_data.drop(['type', 'posture_coding', 'posture2'], axis=1), walk_data[walk_y_var])
sm_classifier = random_forest(sm_data.drop(['type', 'posture_coding'], axis=1), sm_data[sm_y_var])
walk_intensity_classifier = random_forest(walk_intensity_data.drop(['type', 'posture_coding'], axis=1), 
                                          walk_intensity_data[walk_intensity_y_var])

/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/samhsu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:247: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [60]:
save_pkl(stand_classifier, "tier_wrist_stand.sav")
save_pkl(walk_classifier, "tier_wrist_walk.sav")
save_pkl(sm_classifier, "tier_wrist_sm.sav")
save_pkl(walk_intensity_classifier, "tier_wrist_winten.sav")

Classifier saved as tier_wrist_stand.sav
Classifier saved as tier_wrist_walk.sav
Classifier saved as tier_wrist_sm.sav
Classifier saved as tier_wrist_winten.sav


In [59]:
data_with_predictions.shape

(346060, 98)

In [60]:
data.shape

(346060, 97)

In [62]:
data2 = pd.DataFrame(sc.get_complete(user,input_file))
data2 = get_lags(data2, monitor, 1)

In [63]:
data2.shape

(354912, 112)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,thigh.mean.vm,thigh.sd.vm,thigh.mean.ang,thigh.sd.ang,thigh.p625,thigh.dfreq,thigh.ratio.df,...,last.1.chest.mean.x,last.1.chest.mean.y,last.1.chest.mean.z,last.1.chest.sd.x,last.1.chest.sd.y,last.1.chest.sd.z,last.1.chest.mean.xy,last.1.chest.mean.yz,last.1.chest.mean.xz,last.1.chest.mean.xyz
0,0.0,0.0,0.0,0.986613,0.004449,-68.851526,0.481218,0.138523,9.375000,0.149561,...,0.064668,0.872816,0.492027,0.006366,0.007159,0.011546,0.056458,0.429436,0.031795,0.027758
1,1.0,1.0,1.0,0.987922,0.009588,-69.207153,0.862855,0.069721,15.625000,0.154061,...,0.064668,0.872816,0.492027,0.006366,0.007159,0.011546,0.056458,0.429436,0.031795,0.027758
2,2.0,2.0,2.0,0.993664,0.028643,-74.283003,4.769111,0.226041,2.083333,0.153646,...,0.053767,0.865618,0.505544,0.018782,0.020276,0.025370,0.046592,0.437685,0.027058,0.023451
3,3.0,3.0,3.0,0.989123,0.017704,-78.747967,1.705623,0.046754,3.125000,0.135782,...,0.000412,0.906554,0.430838,0.017890,0.032787,0.053736,0.000418,0.389161,0.000151,0.000155
4,4.0,4.0,4.0,0.991041,0.032970,-75.473418,2.917226,0.141949,3.125000,0.140582,...,0.029809,0.910572,0.415069,0.015330,0.007882,0.015396,0.027198,0.377912,0.012216,0.011145
5,5.0,5.0,5.0,0.991524,0.008406,-70.810816,1.520598,0.207626,1.041667,0.138983,...,0.050996,0.893721,0.448203,0.026227,0.018166,0.024315,0.045313,0.400348,0.023391,0.020773
6,6.0,6.0,6.0,0.988216,0.007815,-69.996036,0.612164,0.054837,11.160714,0.116277,...,0.062556,0.863806,0.505276,0.014522,0.009856,0.015992,0.054063,0.436391,0.031582,0.027290
7,7.0,7.0,7.0,0.987351,0.006773,-70.156364,0.530316,0.158569,4.166667,0.144990,...,0.059159,0.872328,0.492515,0.007787,0.006963,0.012122,0.051619,0.429598,0.029089,0.025379
8,8.0,8.0,8.0,0.988761,0.004861,-70.164253,0.409125,0.140228,8.333333,0.126436,...,0.058366,0.873037,0.489635,0.004483,0.004253,0.006858,0.050960,0.427466,0.028584,0.024957
9,9.0,9.0,9.0,0.988466,0.005950,-69.982850,0.702234,0.119028,8.333333,0.132564,...,0.058976,0.870863,0.493843,0.005477,0.004564,0.010461,0.051359,0.430052,0.029131,0.025367
